In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import pandas as pd
import os
from IPython.display import display, clear_output
import ipywidgets as widgets 
import io 

In [ ]:
# --- 경로 및 설정 ---
data_dir = 'safebooru\data'
model_save_path = os.path.join(data_dir, 'model/best_model.pth')

# 태그 이름을 가져오기 위해 train.csv 사용
train_csv_path = os.path.join(data_dir, 'train.csv') 

In [3]:
# --- 장치 설정 ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ 설정 완료, 사용 장치: {device}")

✅ 설정 완료, 사용 장치: cuda


In [4]:
# 태그 이름(클래스) 목록 로드
train_df = pd.read_csv(train_csv_path)
tag_names = [col for col in train_df.columns if col not in ['id', 'created_at', 'rating', 'score', 'sample_url', 'sample_width', 'sample_height', 'preview_url']]
num_tags = len(tag_names)

# 모델 구조 정의 및 가중치 로드
model = models.resnet50()
model.fc = nn.Linear(model.fc.in_features, num_tags)
model.load_state_dict(torch.load(model_save_path, map_location=device))
model = model.to(device)
model.eval()

print(f"✅ 모델 및 {num_tags}개 태그 이름 로드 완료")

✅ 모델 및 278개 태그 이름 로드 완료


In [5]:
def predict_tags(model, device, transform, tag_names, image_path=None, image_bytes=None, threshold=0.5):
    """
    단일 이미지의 태그를 예측하고, 예측된 태그 리스트와 PIL 이미지 객체를 반환합니다.
    """
    if image_path:
        try:
            image = Image.open(image_path).convert("RGB")
        except FileNotFoundError:
            return f"오류: '{image_path}' 파일을 찾을 수 없습니다.", None
    elif image_bytes:
        image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
    else:
        return "오류: 예측할 이미지가 제공되지 않았습니다.", None

    # 이미지 전처리 및 예측
    image_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(image_tensor)
        preds = torch.sigmoid(outputs) > threshold
    
    predicted_indices = preds[0].nonzero(as_tuple=True)[0]
    predicted_tags = [tag_names[i] for i in predicted_indices]
    
    # 예측된 태그 리스트와 이미지 객체를 반환
    return predicted_tags, image

# 추론에 사용할 이미지 변환 (수정 없음)
inference_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

print("✅ 추론 함수 정의 완료")

✅ 추론 함수 정의 완료


In [6]:
# 1. 파일 업로드 위젯과 결과를 표시할 출력 위젯을 만듭니다.
uploader = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='이미지 업로드'
)
out = widgets.Output() # 결과를 표시할 전용 공간

# 2. 파일이 업로드되면 실행될 함수를 수정합니다.
def on_upload_change(change):
    if not change['new']:
        return
    
    with out:
        clear_output(wait=True)
        
        uploaded_file_info = change['owner'].value[0]
        image_bytes = uploaded_file_info['content']

        # ★★★ 예측 함수로부터 결과(태그, 이미지)를 받아옴 ★★★
        predicted_tags, image_to_display = predict_tags(
            model=model,
            device=device,
            transform=inference_transform,
            tag_names=tag_names,
            image_bytes=image_bytes,
            threshold=0.5
        )

        # ★★★ 받아온 결과를 여기서 직접 출력 ★★★
        if image_to_display:
            print("--- 🖼️ 입력된 이미지 🖼️ ---")
            display(image_to_display.resize((224, int(224 * image_to_display.height / image_to_display.width))))
        
        print("\n--- 🚀 예측된 태그 🚀 ---")
        if isinstance(predicted_tags, list) and predicted_tags:
            tags_per_line = 5
            for i in range(0, len(predicted_tags), tags_per_line):
                print("  ".join(predicted_tags[i:i+tags_per_line]))
        elif isinstance(predicted_tags, list):
            print("예측된 태그가 없습니다.")
        else:
            print(predicted_tags) # 오류 메시지 출력
        
        with uploader.hold_sync():
            uploader.value = ()

# 3. 위젯의 변경을 감지하도록 설정합니다.
uploader.observe(on_upload_change, names='value')

# 4. 업로드 버튼과 출력 영역을 함께 표시합니다.
display(uploader, out)

FileUpload(value=(), accept='image/*', description='이미지 업로드')

Output()